# Bibliotecas

In [ ]:
!pip install basedosdados

In [ ]:
import basedosdados as bd

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
#biblioteca para importar a base dados localmente.

# Base de Dados

In [ ]:
bd.list_datasets()
#Essa função lista todos os datasets disponíveis na biblioteca
#No nosso caso serve para darmos acesso da nossa conta do Google Cloud
#Ao rodar abra o link > dê acesso a sua conta > copia o código que eles disponibilizam > cole aqui e aperte enter

In [ ]:
df_IPS = bd.read_table(dataset_id='br_rj_rio_de_janeiro_ipp_ips', 
            table_id='indicadores',
            billing_project_id='<ID do projeto do google cloud>')

#Baixa o banco de dados, dos indicadores do IPS, da base de dados
#Crie um projeto no Google Cloud > https://console.cloud.google.com/projectselector2/home/dashboard
#Cole o Id do projeto no billing_project_id

In [ ]:
df_IPS.to_csv('IPS.csv', sep=',', index=False)
#transforma em formato csv a base de dados.

In [ ]:
files.download('IPS.csv')
#faz o download da base de dados, em formato csv.

# Análise exploratória

In [ ]:
df_IPS.columns

In [ ]:
df_IPS.describe()

In [ ]:
df_IPS.info()

## Acesso água canalizada

In [ ]:
agua_canalizada = df_IPS[['regiao_administrativa', 'ano', 'prop_acesso_agua_canalizada']]
agua_canalizada.sort_values(by = 'prop_acesso_agua_canalizada', ascending=False).head(21)

#O nível de pessoas com acesso a água canalizada é grande. Um pouco menor em Guaratiba só
#Ao longo dos anos nada mudou no Rio como um todo. Mudou em algumas partes, mas a mudança é tão pequena que não reflete na cidade inteira
#Méier não aparece entre os piores e nem entre os melhores

## Adensamento

In [ ]:
pd.DataFrame(df_IPS['regiao_administrativa'].unique()).sort_values(0)adensamento = df_IPS[['regiao_administrativa', 'ano', 'prop_adensamento_habitacional_excessivo']]
(
    adensamento
    .sort_values(by = 'regiao_administrativa')
    .head(60)
)

## Homicídios por morte polícial

In [ ]:
homicidios_policia = df_IPS[['regiao_administrativa', 'ano', 'taxa_homicidios_acao_policial']]
(
    homicidios_policia
    .sort_values(by='regiao_administrativa', ascending=True)
    .head(60)
)

#Na Pavuna caiu bastante de 2018 para 2020, provavelmente pela determinação do STF que proibi ações policiais em favelas
#Muito discrepante as regiões mais ricas para as mais pobres

## Homicidios de jovens negros

In [ ]:
homicidios_negros = df_IPS[['regiao_administrativa', 'ano', 'taxa_homicidios_jovens_negros']]
(
    homicidios_negros[(homicidios_negros['ano']==2018)]
    .sort_values(by='taxa_homicidios_jovens_negros', ascending=True)
    .head(60)
)

#2 regiões da zona sul estão zeradas. Provavelmente os dados estão corretos, pois outras regiões da zona sul os dados são quase zerados
#Não deve ter tantos negros na zona sul e a taxa de homicídios lá já é bem baixa
#Na média a taxa não é tão grande no Méier

## Homicídios

In [ ]:
homicidios = df_IPS[['regiao_administrativa', 'ano', 'taxa_homicidios']]
(
    homicidios
    .sort_values(by='regiao_administrativa', ascending=True)
    .head(60)
)

#Tem um grande salto em todas as regiões administrativas de 2016 para 2020
#Regiões com mais homicídios -> Pavuna, São Cristovão e Anchieta (tudo em 2018)
#Na média dos 3 anos Pavuna, Portuaria e Rio Comprido tem as maiores taxas
#É muito discrepante Copacabana pra Pavuna

## Roubos de rua

In [ ]:
roubos = df_IPS[['regiao_administrativa', 'ano', 'taxa_roubos_rua']]
(
    roubos
    .sort_values(by='taxa_roubos_rua', ascending=False)
    .head(60)
)

#As regiões do centro são as que tem mais assalto de rua

## Acesso eletricidade

In [ ]:
eletricidade = df_IPS[['regiao_administrativa', 'ano', 'prop_acesso_energia_eletrica']]
(
    eletricidade[(eletricidade['ano']==2020)]
    .sort_values(by = 'prop_acesso_energia_eletrica')
    .head(33)

)

#Complexo do alemão é o único com menos de 70%
#Achei que a Rocinha, por ser a maior favela, apareceria entre os primeiros
#Complexo do Alemão e Guaratiba são as regiões com pior desempanho até aqui
#Meier, novamente, no meio pra baixo nas comparações

# Alterações na base

## Criando coluna com IDs das regições administrativas
Com base nos IDs do arquivo dos limites dos bairros, para plotar os mapas

In [ ]:
codra = {
    'Rio De Janeiro':0,
    'Portuaria':1,
    'Centro':2,
    'Rio Comprido':3,
    'Botafogo':4,
    'Copacabana':5,
    'Lagoa':6,
    'São Cristovão':7,
    'Tijuca':8,
    'Vila Isabel':9, 
    'Ramos':10, 
    'Penha':11, 
    'Inhaúma':12, 
    'Méier':13, 
    'Iraja':14,
    'Madureira':15, 
    'Jacarepaguá':16, 
    'Bangu':17, 
    'Campo Grande':18, 
    'Santa Cruz':19,
    'Ilha Do Governador':20, 
    'Anchieta':22, 
    'Santa Teresa':23,
    'Barra Da Tijuca':24, 
    'Pavuna':25, 
    'Guaratiba':26, 
    'Rocinha':27, 
    'Jacarezinho':28,
    'Complexo Do Alemão':29, 
    'Maré':30, 
    'Vigário Geral':31, 
    'Realengo':33,
    'Cidade De Deus':34
    }

In [ ]:
def cria_codra(alguma_coisa):
    for chave, valor in codra.items():
        if chave == alguma_coisa:
            return valor

In [ ]:
df_IPS['codra'] = df_IPS['regiao_administrativa'].apply(codra)

In [ ]:
df_IPS[['regiao_administrativa', 'codra']].head(50)

## Criando coluna das regiões de planejamento

In [ ]:
area_planejamento = {
    'Rio De Janeiro': ['Rio De Janeiro'],
    'Centro': ['Portuaria', 'Centro', 'Rio Comprido', 'São Cristovão', 'Santa Teresa'],
    'Zona Sul': ['Botafogo', 'Copacabana', 'Lagoa', 'Rocinha'],
    'Tijuca': ['Tijuca', 'Vila Isabel'],
    'Ramos': ['Ramos', 'Maré'],
    'Méier': ['Méier', 'Jacarezinho'],
    'Madureira': ['Madureira', 'Iraja'],
    'Inhaúna': ['Inhaúma', 'Complexo Do Alemão'],
    'Penha': ['Penha', 'Vigário Geral'],
    'Pavuna': ['Pavuna', 'Anchieta'],
    'Ilha do Governador': ['Ilha Do Governador'], 
    'Jacarepagua': ['Jacarepaguá', 'Cidade De Deus'],
    'Barra da Tijuca': ['Barra Da Tijuca'],
    'Bangu': ['Bangu', 'Realengo'],
    'Campo Grande': ['Campo Grande'],
    'Santa Cruz': ['Santa Cruz'],
    'Guaratiba': ['Guaratiba'],
}

In [ ]:
#Função que mapeia o dicionário e retorna uma series com os valores da coluna
def cria_area_planejamento(alguma_coisa):
    for key, value in area_planejamento.items():
        for outra_coisa in value:
            if outra_coisa == alguma_coisa:
                return key


In [ ]:
#Cria coluna area de planejamento
df_IPS['area_planejamento'] = df_IPS['regiao_administrativa'].apply(cria_area_planejamento)

In [ ]:
df_IPS[['regiao_administrativa', 'area_planejamento']].head(30)